In [1]:
FN_MODEL_OUTPUT = {'MLP':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy',
                   'RPN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy',
                   'CNN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy',
                   # 'cVAE': './model_outputs/cvae_preds_bestcrps.h5',
                   'cVAE': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/cvae.h5',
                   'HSR': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/hsr_preds_bestcrps.h5',
                  }

In [2]:
# model name
# (model name is used for the output)
model_name = 'MLP'

# input of validation dataset (npy)
fn_x_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_input_stride6.npy'

# true output of validation dataset (npy)
fn_y_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'

# Model predicted output of varlidation dataset (npy)
fn_y_pred = FN_MODEL_OUTPUT[model_name]

# model grid information (nc)
fn_grid = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/E3SM-MMF_ne4_grid-info.orig.nc'

# normalization scale factors (nc)
fn_mli_mean  = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_mean.nc'
fn_mli_min   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_min.nc'
fn_mli_max   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_max.nc'
fn_mlo_scale = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mlo_scale.nc'

# fn_save_output
fn_save_metrics = f'./metrics/{model_name}.metrics.csv'
fn_save_metrics_avg = f'./metrics/{model_name}.metrics.lev-avg.csv'

In [3]:
# physical constatns from (E3SM_ROOT/share/util/shr_const_mod.F90)
grav    = 9.80616    # acceleration of gravity ~ m/s^2
cp      = 1.00464e3  # specific heat of dry air   ~ J/kg/K
lv      = 2.501e6    # latent heat of evaporation ~ J/kg
lf      = 3.337e5    # latent heat of fusion      ~ J/kg
ls      = lv + lf    # latent heat of sublimation ~ J/kg
rho_air = 101325./ (6.02214e26*1.38065e-23/28.966) / 273.15 # density of dry air at STP  ~ kg/m^3
                                                            # ~ 1.2923182846924677
                                                            # SHR_CONST_PSTD/(SHR_CONST_RDAIR*SHR_CONST_TKFRZ)
                                                            # SHR_CONST_RDAIR   = SHR_CONST_RGAS/SHR_CONST_MWDAIR
                                                            # SHR_CONST_RGAS    = SHR_CONST_AVOGAD*SHR_CONST_BOLTZ
rho_h20 = 1.e3       # density of fresh water     ~ kg/m^ 3

vars_mlo_energy_conv = {'ptend_t':cp,
                        'ptend_q0001':lv,
                        'cam_out_NETSW':1.,
                        'cam_out_FLWDS':1.,
                        'cam_out_PRECSC':lv*rho_h20,
                        'cam_out_PRECC':lv*rho_h20,
                        'cam_out_SOLS':1.,
                        'cam_out_SOLL':1.,
                        'cam_out_SOLSD':1.,
                        'cam_out_SOLLD':1.
                       }

In [4]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from data_utils import *

# set dimemsion names for xarray datasets
dim_name_level  = 'lev'
dim_name_sample = 'sample'

In [5]:
# load input dataset
x_true = np.load(fn_x_true).astype(np.float64)
y_true = np.load(fn_y_true).astype(np.float64)
if fn_y_pred[-3:] == '.h5':
    y_pred = xr.open_dataset(fn_y_pred)['pred'].values
else:
    y_pred = np.load(fn_y_pred).astype(np.float64)
N_samples = y_pred.shape[0] 

# load norm/scale factors
mlo_scale = xr.open_dataset(fn_mlo_scale)
mli_mean  = xr.open_dataset(fn_mli_mean)
mli_min   = xr.open_dataset(fn_mli_min)
mli_max   = xr.open_dataset(fn_mli_max)

In [6]:
# load grid information
ds_grid = xr.open_dataset(fn_grid) # has ncol:384
N_ncol = len(ds_grid['ncol']) # length of ncol dimension (nlat * nlon)

# make area-weights
ds_grid['area_wgt'] = ds_grid['area'] / ds_grid['area'].mean('ncol')

# map ds_grid's ncol dimension -> the N_samples dimension of npy-loayd arrays (e.g., y_pred)
to_xarray = {'area_wgt': (dim_name_sample,np.tile(ds_grid['area_wgt'], int(N_samples/len(ds_grid['ncol'])))),
            }
to_xarray = xr.Dataset(to_xarray)

# add nsample-mapped grid variables back to ds_grid
ds_grid = xr.merge([ds_grid  [['P0', 'hyai', 'hyam','hybi','hybm','lat','lon','area']],
                    to_xarray[['area_wgt']]])

In [7]:
# list of ML output variables
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC',
            'cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD'] # mlo mean ML output.

# length of each variable
vars_mlo_len = {'ptend_t':60,
                'ptend_q0001':60,
                'cam_out_NETSW':1,
                'cam_out_FLWDS':1,
                'cam_out_PRECSC':1,
                'cam_out_PRECC':1,
                'cam_out_SOLS':1,
                'cam_out_SOLL':1,
                'cam_out_SOLSD':1,
                'cam_out_SOLLD':1
               }

# map the length of dimension to the name of dimension
len_to_dim = {60:dim_name_level,
              N_samples: dim_name_sample}

In [8]:
# Here, we first construct a dictionary of {var name: (dimension name, array-like)},
# then, map the dictionary to an Xarray Dataset.
# (ref: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

DS = {}

for kds in ['true', 'pred']:
    if kds=='true':
        work = y_true
    elif kds=='pred':
        work = y_pred

    # [1] Construct dictionary for xarray dataset
    #     format is key for variable name /
    #               value for a turple of (dimension names, data).
    to_xarray = {}
    for k, kvar in enumerate(vars_mlo):

        # length of variable (ie, number of levels)
        kvar_len = vars_mlo_len[kvar]

        # set dimensions of variable
        if kvar_len == 60:
            kvar_dims = (dim_name_sample, dim_name_level)
        elif kvar_len == 1:
            kvar_dims = dim_name_sample

        # set start and end indices of variable in the loaded numpy array
        # then, add 'kvar':(kvar_dims, <np_array>) to dictionary
        if k==0: ind1=0
        ind2 = ind1 + kvar_len

        # scaled output
        kvar_data = np.squeeze(work[:,ind1:ind2])
        # unscaled output
        kvar_data = kvar_data / mlo_scale[kvar].values

        to_xarray[kvar] = (kvar_dims, kvar_data)

        ind1 = ind2

    # [2] convert dict to xarray dataset
    DS[kds] = xr.Dataset(to_xarray)

    # [3] add surface pressure ('state_ps') from ml input
    # normalized ps
    state_ps =  xr.DataArray(x_true[:,120], dims=('sample'), name='state_ps')
    # denormalized ps
    state_ps = state_ps * (mli_max['state_ps'] - mli_min['state_ps']) + mli_mean['state_ps']
    DS[kds]['state_ps'] = state_ps

    # [4] add grid information
    DS[kds] = xr.merge([DS[kds], ds_grid])

    # [5] add pressure thickness of each level, dp
    # FYI, in a hybrid sigma vertical coordinate system, pressure at level z is
    # P[x,z] = hyam[z]*P0 + hybm[z]*PS[x,z],
    # where, hyam and hybm are 
    tmp = DS[kds]['P0']*DS[kds]['hyai'] + DS[kds]['state_ps']*DS[kds]['hybi']
    tmp = tmp.isel(ilev=slice(1,61)).values - tmp.isel(ilev=slice(0,60)).values
    tmp = tmp.transpose()
    DS[kds]['dp'] = xr.DataArray(tmp, dims=('sample', 'lev'))

    # [6] break (sample) to (ncol,time)
    N_timestep = int(N_samples/N_ncol)
    dim_ncol     = np.arange(N_ncol)
    dim_timestep = np.arange(N_timestep)
    new_ind = pd.MultiIndex.from_product([dim_timestep, dim_ncol],
                                         names=['time', 'ncol'])
    DS[kds] = DS[kds].assign_coords(sample=new_ind).unstack('sample')

In [9]:
# [1] Weight vertical levels by dp/g that is equivalent to a mass of air within a grid cell per unit area [kg/m2]
# [2] Weight horizontal area of each grid cell by a[x]/mean(a[x]).
# [3] Unit conversion to a common energy unit

DS_ENERGY = {}
for kds in ['true','pred']:
    # Make a copy to keep original dataset
    DS_ENERGY[kds] = DS[kds].copy(deep=True)

    # vertical weighting / area weighting / unit conversion
    for kvar in vars_mlo:

        # [1] weight vertical levels by dp/g
        #     ONLY for vertically-resolved variables, e.g., ptend_{t,q0001}
        # dp/g = - \rho * dz
        if vars_mlo_len[kvar] == 60:
            DS_ENERGY[kds][kvar] = DS_ENERGY[kds][kvar] * DS_ENERGY[kds]['dp']/grav

        # [2] weight area
        #     for ALL variables
        DS_ENERGY[kds][kvar] = DS_ENERGY[kds]['area_wgt'] * DS_ENERGY[kds][kvar]

        # [3] convert units to W/m2
        #     for variables with different units, e.g., ptend_{t,q0001}, precsc, precc
        DS_ENERGY[kds][kvar] =  vars_mlo_energy_conv[kvar] * DS_ENERGY[kds][kvar]

In [65]:
vars_mlo[0]

'ptend_t'

In [86]:
np.sum((DS['true']['ptend_t'].values[:,1,:]*mlo_scale['ptend_t'].values[:,np.newaxis]).transpose() - y_true[384:384+384,0:60])

9.419429889941447e-18

In [91]:
jerrymander = y_true[:,:60].reshape((int(1681920/384), 384, 60))

In [93]:
jerrymander.shape

(4380, 384, 60)

In [105]:
y_true.shape

(1681920, 128)

In [184]:
mlo_scale['ptend_t'].values

array([1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64, 1004.64,
       1004.64, 1004.64, 1004.64, 1004.64])

In [193]:
y_true[:,:60].reshape((int(1681920/384), 384, 60)).shape

(4380, 384, 60)

In [187]:
y_true[:,60:120].reshape((int(1681920/384), 384, 60)).shape

(4380, 384, 60)

In [208]:
y_true[:,120].reshape((int(1681920/384), 384)).shape

(4380, 384)

In [198]:
mlo_scale['ptend_t'].values[np.newaxis, np.newaxis, :].shape

(1, 1, 60)

In [204]:
mlo_scale['cam_out_NETSW'].values

array(0.0024)

In [205]:
area_wgt

NameError: name 'area_wgt' is not defined

# Unit testing

In [142]:
vars_mlo

['ptend_t',
 'ptend_q0001',
 'cam_out_NETSW',
 'cam_out_FLWDS',
 'cam_out_PRECSC',
 'cam_out_PRECC',
 'cam_out_SOLS',
 'cam_out_SOLL',
 'cam_out_SOLSD',
 'cam_out_SOLLD']

In [103]:
np.sum(y_true[:,:60].reshape((int(1681920/384), 384, 60))[1,:,:] - y_true[384:384+384,0:60])

0.0

In [138]:
np.sum((DS['true']['ptend_t'].values[:,1,:]*mlo_scale['ptend_t'].values[:,np.newaxis]).transpose() - y_true[384:384+384,0:60])

9.419429889941447e-18

In [104]:
np.sum(y_true[:,60:120].reshape((int(1681920/384), 384, 60))[1,:,:] - y_true[384:384+384,60:120])

0.0

In [143]:
np.sum((DS['true']['ptend_q0001'].values[:,1,:]*mlo_scale['ptend_q0001'].values[:,np.newaxis]).transpose() - y_true[384:384+384,60:120])

1.1922414029637306e-16

In [107]:
np.sum(y_true[:,120].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,120])

0.0

In [163]:
np.sum(DS['true']['cam_out_NETSW'].values[1,:]*mlo_scale['cam_out_NETSW'].values - y_true[384:384+384,120])

-7.563394355258879e-16

In [108]:
np.sum(y_true[:,121].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,121])

0.0

In [164]:
np.sum(DS['true']['cam_out_FLWDS'].values[1,:]*mlo_scale['cam_out_FLWDS'].values - y_true[384:384+384,121])

0.0

In [109]:
np.sum(y_true[:,122].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,122])

0.0

In [ ]:
np.sum(DS['true']['cam_out_FLWDS'].values[1,:]*mlo_scale['cam_out_FLWDS'].values - y_true[384:384+384,121])

In [110]:
np.sum(y_true[:,123].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,123])

-5.684341886080802e-14

In [111]:
np.sum(y_true[:,124].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,124])

4.547473508864641e-13

In [126]:
np.sum(y_true[:,125].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,125])

-4.661160346586257e-12

In [127]:
np.sum(y_true[:,126].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,126])

0.0

In [128]:
np.sum(y_true[:,127].reshape((int(1681920/384), 384, 1))[1,:,:] - y_true[384:384+384,127])

0.0

In [173]:
np.sum(y_true[:,120:].reshape((int(1681920/384), 384, -1))[1,:,:] - y_true[384:384+384,120:])

0.0

In [179]:
scalarmander = y_true[:, 120:].reshape((int(1681920/384), 384, -1))[1,:,:]
scalarmander.shape

(384, 8)

In [181]:
scalarmander = y_true[:, 120:].reshape((int(1681920/384), 384, -1))[1,:,:]
np.sum(scalarmander-y_true[384:384+384,120:])

0.0

In [165]:
y_true[:,120:].reshape((int(1681920/384), 384, -1)).shape

(4380, 384, 8)

In [167]:
y_true[:,60:120].reshape((int(1681920/384), 384, 60)).shape

(4380, 384, 60)

In [137]:
np.sum((DS['true']['ptend_t'].values[:,1,:]*mlo_scale['ptend_t'].values[:,np.newaxis]).transpose() - y_true[384:384+384,0:60])

9.419429889941447e-18

In [134]:
y_true[:,120:].reshape((int(1681920/384), 384, -1)).shape

(4380, 384, 8)

In [136]:
np.sum(y_true[:,120:].reshape((int(1681920/384), 384, -1))[1,:,:] - y_true[384:384+384,120:])

0.0

In [169]:
scalarmander = y_true[:, 120:].reshape((int(1681920/384), 384, -1))
np.sum(scalarmander[1,:,1]-y_true[384:384+384,120])

518.2811775077098

In [99]:
hmm3 = y_true[:, 120:].reshape((int(1681920/384), 384, 8))

In [98]:
np.sum(scalarmander[1,:,1]-y_true[384:384+384,120])

518.2811775077098

In [168]:
scalarmander[1,:,1].shape

(384,)

In [90]:
y_true.shape[0]

1681920

In [58]:
hakd = y_true[:, 0:60].reshape((4380, 384, 60)).transpose((2,0,1))/mlo_scale['ptend_t'].values[:,np.newaxis,np.newaxis]
hakd.shape

(60, 4380, 384)

In [13]:
y_pred[:, 0:60].reshape((4380, 384, 60)).transpose((2,0,1)).shape

(60, 4380, 384)

In [14]:
pred_ex = np.array(DS_ENERGY['pred'][vars_mlo[2]])
pred_ex.shape

(4380, 384)

In [15]:
wowza = y_pred[:,120:].reshape((4380, 384, -1)).transpose((2, 0, 1))
wowza.shape

(8, 4380, 384)

In [16]:
wak = y_pred[:,120]

In [31]:
state_ps_alt = x_true[:,120]*(mli_max['state_ps'].values - mli_min['state_ps'].values) + mli_mean['state_ps'].values
state_ps_alt_reshaped = np.reshape(state_ps_alt, (-1,384))

In [32]:
state_ps_alt.shape

(1681920,)

In [33]:
state_ps_alt_reshaped.shape

(4380, 384)

In [34]:
hmm2 = np.array(ds_grid['P0']*ds_grid['hyai'])[:,np.newaxis,np.newaxis] + ds_grid['hybi'].values[:, np.newaxis, np.newaxis] * state_ps_alt_reshaped[np.newaxis, :, :]

In [35]:
hmm2.shape

(61, 4380, 384)

In [36]:
lol = hmm2[1:61,:,:] - hmm2[0:60,:,:]

In [37]:
dp_ref = DS[kds]['dp'].values

In [38]:
np.sum(lol-dp_ref)

0.0

In [39]:
dp_ref.shape

(60, 4380, 384)

In [40]:
ds_grid['area_wgt'].shape

(1681920,)

In [41]:
DS['true']['area_wgt']

<xarray.DataArray 'area_wgt' (time: 4380, ncol: 384)>
array([[0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       ...,
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383

In [42]:
okeh = y_pred[:, 0:60].shape

In [43]:
okeh

(1681920, 60)

In [44]:
DS['true']['dp']

<xarray.DataArray 'dp' (lev: 60, time: 4380, ncol: 384)>
array([[[   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        ...,
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817]],

       [[   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
        [   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
        [   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
...
        [1459.9266813 , 1471.22505066, 1430.42324241, ...,
         1385.27574414, 1465.39475656, 1464.28781425],
        [1458.73305834, 1470.35831927, 1429.63945825, ...,
         1385.80204525, 1466.67312371, 1465.17957177],
        [1459.75239868, 1471.00650404, 1431.74712721, ...,
         1387.53251194, 1467.96758963, 1467.26054248]],

       [[1489.97206792, 1502.89740105, 1458.63716644, ...,
         1384.42388415, 1458.37787713, 1460.30758745],
        [1489.4934162 , 1501.23566766, 1458.43904255, ...,
         1384.08881605, 1457.67663963, 1459.64745319],
        [1487.3687768 , 1500.10258501, 1455.3836193 , ...,
         1382.80318621, 1457.16745813, 1458.11426899],
        ...,
        [1481.41055125, 1492.87518424, 1451.47294806, ...,
         1405.66107192, 1486.95909317, 1485.83586141],
        [1480.19936327, 1491.99569827, 1450.67762996, ...,
         1406.19511793, 1488.25627241, 1486.74074177],
        [1481.23370393, 1492.65342155, 1452.81631478, ...,
         1407.95104968, 1489.56978733, 1488.85233547]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * lev      (lev) float64 0.07835 0.1411 0.2529 0.4493 ... 963.6 978.0 992.6

In [45]:
DS['pred']['area_wgt']

<xarray.DataArray 'area_wgt' (time: 4380, ncol: 384)>
array([[0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       ...,
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383

In [46]:
hola = np.array(ds_grid['area_wgt'])
hola.shape

(1681920,)

In [47]:
adios = np.reshape(hola, (-1, 384))

In [48]:
adios.shape

(4380, 384)